# 영어 Word2Vec 만들기

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

### 훈련데이터 이해하기

In [3]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f9fc2cbdd10>)

In [4]:
!ls -l

total 72792
drwxr-xr-x 1 root root     4096 Feb 22 14:38 sample_data
-rw-r--r-- 1 root root 74533638 Feb 24 00:54 ted_en-20160408.xml


### 훈련 데이터 전처리하기
- 실질적 데이터는 영어문장으로만 구성된 내용을 담고 있는 <content>와 </content> 사이의 내용입니다. 
- 전처리 작업을 통해 xml 문법들은 제거하고, 해당 데이터만 가져와야 합니다. 뿐만 아니라, <content>와 </content> 사이의 내용 중에는 (Laughter)나 (Applause)와 같은 배경음을 나타내는 단어도 등장하는데 이 또한 제거해야 합니다.

In [5]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [6]:
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [7]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

In [8]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

In [9]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [10]:
print(f'총 샘플의 개수 : {len(result)}')
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

총 샘플의 개수 : 273424
['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


### Word2Vec 훈련시키기
- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수 (앞뒤 단어의 수)
- sg = 0은 CBOW, 1은 Skip-gram

In [11]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [12]:
model_result = model.wv.most_similar("man")
model_result

[('woman', 0.8525322079658508),
 ('guy', 0.7888501882553101),
 ('lady', 0.7710676789283752),
 ('boy', 0.7561002969741821),
 ('girl', 0.7381208539009094),
 ('soldier', 0.713817834854126),
 ('gentleman', 0.710548460483551),
 ('kid', 0.6719340085983276),
 ('poet', 0.6690990924835205),
 ('friend', 0.6432890295982361)]

In [14]:
model.wv.most_similar(positive=['man', 'girl'], negative=['boy'], topn=3)
# man - boy + girl = woman

[('woman', 0.8422590494155884),
 ('lady', 0.7404364347457886),
 ('guy', 0.7323157787322998)]

###  Word2Vec 모델 저장하고 로드하기

In [15]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장

In [16]:
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드
model_result = loaded_model.most_similar("man") # ! wv가 빠져있음
model_result

[('woman', 0.8525322079658508),
 ('guy', 0.7888501882553101),
 ('lady', 0.7710676789283752),
 ('boy', 0.7561002969741821),
 ('girl', 0.7381208539009094),
 ('soldier', 0.713817834854126),
 ('gentleman', 0.710548460483551),
 ('kid', 0.6719340085983276),
 ('poet', 0.6690990924835205),
 ('friend', 0.6432890295982361)]

In [17]:
!ls -l

total 97976
-rw-r--r-- 1 root root 25786693 Feb 24 01:09 eng_w2v
drwxr-xr-x 1 root root     4096 Feb 22 14:38 sample_data
-rw-r--r-- 1 root root 74533638 Feb 24 00:54 ted_en-20160408.xml
